# Import Interreg NWE Data 
### http://www.nweurope.eu/media/4533/list-of-beneficiaries-for-website.xlsx

In [ ]:
import difflib
import json
import os
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

In [ ]:
all_benefs = pd.read_excel('input/list-of-beneficiaries-for-website.xlsx')
all_benefs.shape

## Load Data

In [ ]:
#all_benefs.columns

In [ ]:
all_benefs.rename({
    'No\nNo\nNr.\nNr.': 'Id',
    'Beneficiary name\nNom du bénéfictiaire\nName des Begünstigten \nNaam van de begunstigde ': 'beneficiary',
    "Operation name\nNom de l'opération\nBezeichnung des Vorhabens\nNaam van de concrete actie": 'project',
    "Operation summary\nRésumé de l'opération\nZusammenfassung des Vorhabens\nSamenvatting van de concrete actie": 'project_summary',
    "Operation start date\nDate de début de l'opération\nDatum des Beginns des Vorhabens\nBegindatum van de concrete actie": 'start_date',
    "Operation end date\nDate de fin de l'opération \nDatum des Endes des Vorhabens \nEinddatum van de concrete actie": 'end_date',
    'Total eligible expenditure allocated to the beneficiary\nTotal des dépenses éligibles attribué au bénéficiaire\nGesamtbetrag der förderfähigen Ausgaben an Begünstigte\nTotale subsidiabele uitgaven toegewezen aan begunstigde': 'funding',
    "Union co-financing rate\nTaux de cofinancement par l'Union \nUnions-Kofinanzierungssatz pro Prioritätsachse\nMedefinancieringspercentage van de Unie (per prioritaire as)": 'union_cofinancing',
    "Operation post code\nCode postal de l\'opération\nPostleitzahl des Vorhabens\nPostcode van de concrete actie": 'raw_postcode',
    'Country\nPays\nLand\nLand': 'country',
    "Name of category of intervention for the operation in accordance with point (b) (vi) of the first subparagraph of Article 96(2)\nDénomination de la catégorie d'intervention dont relève l'opération conformément à l'article 96, paragraphe 2,\npremier alinéa, point b) vi)\nBezeichnung der Interventionskategorie für das Vorhaben gemäß Artikel 96 Absatz 2 Unterabsatz 1 Buchstabe b Ziffer vi;\nNaam van de categorie steunverlening voor de concrete actie, overeenkomstig artikel 96, lid 2, eerste alinea,\nonder b), vi)": 'raw_category',
    'Date of last update of the list of operations\nDate de la dernière mise à jour de la liste des opérations\nDatum der letzten Aktualisierung der Liste der Vorhaben\nDatum van de laatste bijwerking van de lijst van concrete acties': 'last_update'
}, axis=1, inplace=True)

In [ ]:
all_benefs.columns

In [ ]:
all_benefs.shape

## Check Data

In [ ]:
all_benefs.describe()

### `beneficiary`

In [ ]:
[
    all_benefs.beneficiary.isna().sum(),
    (all_benefs.beneficiary.str.strip() != all_benefs.beneficiary).sum()
]

In [ ]:
all_benefs.beneficiary = all_benefs.beneficiary.str.strip()

In [ ]:
[
    all_benefs.beneficiary.nunique(),
    all_benefs.beneficiary.str.lower().nunique()
]

In [ ]:
all_benefs.beneficiary[
    all_benefs.beneficiary.str.lower().duplicated(keep=False) &
    ~all_benefs.beneficiary.duplicated(keep=False)
]

The beneficiaries that are not unique by case are outside the UK, so let's just leave them be.

### `project`

In [ ]:
[
    all_benefs.project.isna().sum(),
    (all_benefs.project.str.strip() != all_benefs.project).sum()
]

In [ ]:
all_benefs.project = all_benefs.project.str.strip()

In [ ]:
[
    all_benefs.project.nunique(),
    all_benefs.project.str.lower().nunique()
]

In [ ]:
raw_unique_projects = all_benefs.project.sort_values().drop_duplicates()
raw_unique_projects

In [ ]:
PROJECT_RX = r'^(.+) - (.+)$'
raw_unique_projects[~raw_unique_projects.str.match(PROJECT_RX)]

`BONE` seems to be duplicated. We will deal with it below.

### `project_summary`

In [ ]:
[
    (~all_benefs.project_summary.isna()).sum(),
    ((~all_benefs.project_summary.isna()) &
     (all_benefs.project_summary.str.strip() != all_benefs.project_summary)).sum()
]

In [ ]:
all_benefs.project_summary = all_benefs.project_summary.str.strip()

In [ ]:
all_benefs.project_summary.nunique()

### `start_date`

In [ ]:
(~all_benefs.start_date.isna()).sum()

In [ ]:
all_benefs.start_date.head()

In [ ]:
all_benefs.start_date = pd.to_datetime(all_benefs.start_date, format='%d.%m.%Y')
all_benefs.start_date.describe()

### `end_date`

In [ ]:
(~all_benefs.end_date.isna()).sum()

In [ ]:
all_benefs.end_date.head()

In [ ]:
all_benefs.end_date = pd.to_datetime(all_benefs.end_date, format='%d.%m.%Y')
all_benefs.end_date.describe()

### `union_cofinancing`

Varies per-partner, not just per-project. Some zeros, but they are all outside the UK, so we can ignore them.

In [ ]:
all_benefs.union_cofinancing.value_counts()

In [ ]:
all_benefs[all_benefs.union_cofinancing < 0.6]

### `funding`

Here things get a bit weird. The column name, "Total eligible expenditure allocated to the beneficiary" suggests to me that this is the budget for the project, and the EU will provide 60% (or whatever the cofinancing rate is). That interpretation matches what we find below from their website UNLESS the project has no start and end date, in which case it seems to have the opposite sense. So we will split up this column here into one for each case.

In [ ]:
all_benefs['contribution_eur'] = all_benefs.funding * all_benefs.union_cofinancing
all_benefs['total_cost_eur'] = all_benefs.funding

is_missing_dates = all_benefs.start_date.isna()
all_benefs.loc[is_missing_dates, 'contribution_eur'] = all_benefs.funding[is_missing_dates]
all_benefs.loc[is_missing_dates, 'total_cost_eur'] = \
    all_benefs.funding[is_missing_dates] / all_benefs.union_cofinancing[is_missing_dates]

In [ ]:
all_benefs.drop(columns=['funding'], inplace=True)

In [ ]:
all_benefs.describe()

### `country`

In [ ]:
(~all_benefs.country.isna()).sum()

In [ ]:
all_benefs.country.sort_values().unique()

### `raw_category`

The inconsistencies here are quite strange. A pattern that sometimes but not always repeats is that the first three partners have leading zeros, but the rest do not.

In [ ]:
all_benefs.loc[
    all_benefs.project == 'SHICC - Sustainable Housing for Inclusive and Cohesive Cities',
    ['Id', 'raw_category']]

In [ ]:
all_benefs.loc[
    all_benefs.project == 'UP-Straw - Urban and Public Buildings in Straw',
    ['Id', 'raw_category']]

And sometimes the dash is missing.

In [ ]:
all_benefs.loc[
    all_benefs.project == 'CHIPS - Cycle Highways Innovation for smarter People Transport and Spatial Planning',
    ['Id', 'raw_category']]

In [ ]:
RAW_CATEGORY_RX = r'^0*(\d+)\s*-?\s*(.+)$'
[
    (~all_benefs.raw_category.isna()).sum(),
    (all_benefs.raw_category.str.strip() != all_benefs.raw_category).sum(),
    all_benefs.raw_category.nunique(),
    all_benefs.raw_category.str.match(RAW_CATEGORY_RX).sum()
]

In [ ]:
all_benefs.raw_category.sort_values().unique()

In [ ]:
all_benefs['category_number'] = all_benefs.raw_category.str.replace(RAW_CATEGORY_RX, r'\1').astype('int')
all_benefs['category_description'] = all_benefs.raw_category.str.replace(RAW_CATEGORY_RX, r'\2')

In [ ]:
all_benefs[['category_number', 'category_description']]. \
    sort_values(['category_number', 'category_description']). \
    drop_duplicates()

So the category text and numbers seem to be very consistent, except for the leading zeros and the dash. Perhaps pulled from dropdowns in different systems?

### `last_update`

Always has the same value. Ignore.

In [ ]:
all_benefs.last_update.unique()

In [ ]:
all_benefs.drop(columns='last_update', inplace=True)

## Extract Projects and Partners

It looks like this is the result of a join on projects and partners. Let's see if we can recover the projects.

In [ ]:
all_projects = all_benefs[[
    'project', 'project_summary', 'start_date', 'end_date',
    'category_number', 'category_description'
]].drop_duplicates()
[raw_unique_projects.shape, all_projects.shape]

In [ ]:
all_projects

Let's remove the duplicate BONE at this point. Also odd: the amounts in `BONE-` are 60% of those above, e.g. `461594.010 / 769323.350 = 0.6`.

In [ ]:
all_benefs[all_benefs.project.str.startswith('BONE')]

In [ ]:
all_projects = all_projects[~all_projects.project.str.startswith('BONE-')]
all_projects.reset_index()
all_projects.insert(0, 'project_id', range(1, all_projects.shape[0] + 1))
all_projects.shape

In [ ]:
all_projects.head()

In [ ]:
all_partnerships = pd.merge(
    all_benefs[[
        'Id', 'beneficiary', 'project',
        'contribution_eur', 'total_cost_eur', 'union_cofinancing',
        'raw_postcode', 'country'
    ]],
    all_projects[['project', 'project_id']]
)
all_partnerships.shape

In [ ]:
all_partnerships.drop(columns='project', inplace=True)
all_partnerships.rename(columns={'Id': 'partner_number'}, inplace=True)
all_partnerships.head(20)

In [ ]:
[
    all_partnerships[['project_id', 'partner_number']].drop_duplicates().shape,
    all_partnerships.shape
]

### Cross Check Partner Numbers

One anomaly (Project ID 45), but the project has no UK partners, so we can ignore it.

In [ ]:
project_num_partnerships = all_partnerships.groupby('project_id').\
    partner_number.count().reset_index().rename(columns={'partner_number': 'num_partners'})
[all_projects.shape[0], project_num_partnerships.shape]

In [ ]:
project_num_partnerships_check = \
    (all_partnerships.reset_index().groupby('project_id').partner_number.max()).\
    reset_index().rename(columns={'partner_number': 'num_partners'})
[all_projects.shape[0], project_num_partnerships_check.shape]

In [ ]:
project_num_partnerships[project_num_partnerships.num_partners != project_num_partnerships_check.num_partners]

In [ ]:
project_num_partnerships_check[project_num_partnerships.num_partners != project_num_partnerships_check.num_partners]

In [ ]:
all_projects[all_projects.project_id == 45]

In [ ]:
all_benefs[all_benefs.project.str.startswith('SeRaMCo')]

### Per-Project Stats

It is useful to get project totals for display and also for comparison with the numbers on their website.

In [ ]:
project_stats = all_partnerships.groupby('project_id').aggregate({
    'contribution_eur': sum,
    'total_cost_eur': sum,
    'country': lambda x: x.unique().shape[0]
}).reset_index().rename(columns={
    'contribution_eur': 'project_contribution_eur',
    'total_cost_eur': 'project_total_cost_eur',
    'country': 'num_countries'
})
project_stats = pd.merge(project_stats, project_num_partnerships)
project_stats

## Restrict to UK Projects

In [ ]:
uk_partnerships = all_partnerships[all_partnerships.country == 'United Kingdom'].copy()
uk_partnerships.shape

In [ ]:
uk_partnerships.head()

In [ ]:
uk_projects = pd.merge(
    all_projects[all_projects.project_id.isin(uk_partnerships.project_id)],
    project_stats
)
uk_projects.shape

In [ ]:
uk_projects.head()

## Find Missing Start and End Dates

They are missing from the export, but they are on the website, albeit in less precise form. Let's get them from the website where we are missing them.

In [ ]:
def fetch_search_results():
    projects = []
    
    def find_project_links(soup):
        for card in soup.findAll('div', {'class': 'project-results-listing__item'}):
            image = card.find('div', {'class': 'project-results-listing__image'}).find('img')
            title = card.find('h2').find('a')
            objective = card.find('p')
            projects.append({
                'project': title.text,
                'page_path': title['href'],
                'image_path': image['src'],
                'objective': objective.text,
            })
            
    project_search_url = 'http://www.nweurope.eu/projects/project-search/'
    for page in range(1, 6):
        print('fetch search page', page)
        response = requests.get(project_search_url, { 'page': page })
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html5lib')
        find_project_links(soup)
        
    return pd.DataFrame(projects)

def fetch_project_info(project_path):
    print('fetch project page', project_path)
    project_url = 'http://www.nweurope.eu' + project_path
    response = requests.get(project_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html5lib')
    
    def find_project_summary():
        return str(soup.find('div', {'class': 'project-summary__content'}))
    
    def find_key_information_item(target_text):
        key_information = soup.find('aside', {'class': 'project-summary__aside'})
        heading = key_information.find('p', text=target_text)
        p_tags = heading.parent.findAll('p')
        assert len(p_tags) == 2
        return p_tags[1].text.strip()
    
    return {
        'eu_funding': find_key_information_item('EU FUNDING'),
        'total_budget': find_key_information_item('TOTAL BUDGET'),
        'timeline': find_key_information_item('TIMELINE'),
        'project_summary': find_project_summary()
    }

def build_website_search_results():
    search_results = fetch_search_results()
    project_info = search_results.page_path.apply(
        lambda path: pd.Series(fetch_project_info(path))
    )
    return pd.merge(search_results, project_info, left_index=True, right_index=True)
    
WEBSITE_SEARCH_RESULTS_PATHNAME = 'output/website_search_results.pkl.gz'
if not os.path.exists(WEBSITE_SEARCH_RESULTS_PATHNAME):
    website_search_results = build_website_search_results()
    website_search_results.to_pickle(WEBSITE_SEARCH_RESULTS_PATHNAME)
    
website_search_results = pd.read_pickle(WEBSITE_SEARCH_RESULTS_PATHNAME)
website_search_results.head()

### Match Project Names

The website project names don't match what we have from the Excel file. Try a fuzzy match and then fix up the ones it gets wrong manually.

In [ ]:
def difflib_get_closest_matching_project(target):
    matches = difflib.get_close_matches(target, website_search_results.project)
    if len(matches) > 0:
        return matches[0]
    else:
        return None
uk_projects['website_project'] = uk_projects.project.apply(difflib_get_closest_matching_project)
uk_projects[['project', 'website_project']]

In [ ]:
uk_projects.loc[uk_projects.project.str.startswith('BE-GOOD'), 'website_project'] = \
    'BEGOOD'
uk_projects.loc[uk_projects.project.str.startswith('E = 0'), 'website_project'] = \
    'E=0: Desirable, warm, affordable homes for life'
uk_projects.loc[uk_projects.project.str.startswith('FIBERSORT'), 'website_project'] = \
    'Bringing the Fibersort technology to the market'
uk_projects.loc[uk_projects.project.str.startswith('GENCOMM'), 'website_project'] = \
    'GENCOMM: GENerating energy secure COMMunities'
uk_projects.loc[uk_projects.project.str.startswith('PowerVIBES'), 'website_project'] = \
    'PowerVIBES - Sustainable solution for the festival market'
uk_projects.loc[uk_projects.project.str.startswith('LL4WIDE'), 'website_project'] = \
    'Water Test Network' # the LL4WIDE page is a 404, but a Google Cached Version had the same content as WTN
uk_projects.loc[uk_projects.project.str.startswith('UNEET'), 'website_project'] = \
    'UNEET - PROFESSIONAL INTEGRATION OF NW EUROPEAN YOUNG ADULTS INTO THE HOTEL, RESTAURANT AND CATERING SECTOR'
    
uk_projects[['project', 'website_project']]

In [ ]:
uk_projects_with_website_info = pd.merge(
    uk_projects, website_search_results,
    left_on='website_project', right_on='project', suffixes=('', '_website'))
[uk_projects_with_website_info.shape, uk_projects.shape]

In [ ]:
uk_projects_with_website_info.head()

### Check Funding and Budget Amounts

With the logic mentioned above to interpret `funding` as either EU contribution or budget, we get pretty close. There is one project that is 12% out (Passion HF), but the rest are <3% out.

In [ ]:
def parse_website_eur(frame, column):
    RX = r'^€ ([\d.]+) m$'
    assert frame[column].str.match(RX).all()
    frame[column + '_eur'] = \
        frame[column].str.replace(RX, r'\1').astype('float') * 1e6
    
parse_website_eur(uk_projects_with_website_info, 'eu_funding')
parse_website_eur(uk_projects_with_website_info, 'total_budget')

uk_projects_with_website_info[['eu_funding', 'eu_funding_eur', 'total_budget', 'total_budget_eur']].head()

In [ ]:
uk_projects_with_website_info['delta_contribution_eur'] = \
    (uk_projects_with_website_info.project_contribution_eur - \
     uk_projects_with_website_info.eu_funding_eur).abs() / \
    uk_projects_with_website_info.eu_funding_eur
uk_projects_with_website_info['delta_total_cost_eur'] = \
    (uk_projects_with_website_info.project_total_cost_eur - \
     uk_projects_with_website_info.total_budget_eur).abs() / \
    uk_projects_with_website_info.total_budget_eur
uk_projects_with_website_info[['delta_contribution_eur', 'delta_total_cost_eur']].describe()

In [ ]:
uk_projects_with_website_info[uk_projects_with_website_info.delta_contribution_eur > 0.05]

### Parse Start and End Dates

In [ ]:
def parse_website_timeline(frame):
    RX = r'^(\d+)-(\d+)$'
    assert frame.timeline.str.match(RX).all()
    frame['website_start_date'] = pd.to_datetime(frame.timeline.str.replace(RX, r'\1-01-01'))
    frame['website_end_date'] = pd.to_datetime(frame.timeline.str.replace(RX, r'\2-12-31'))
parse_website_timeline(uk_projects_with_website_info)
uk_projects_with_website_info[['timeline', 'start_date', 'website_start_date', 'end_date', 'website_end_date']]

In [ ]:
def fill_missing_dates(frame):
    is_missing_dates = frame.start_date.isna()
    frame.loc[is_missing_dates, 'start_date'] = frame.website_start_date[is_missing_dates]
    frame.loc[is_missing_dates, 'end_date'] = frame.website_end_date[is_missing_dates]
fill_missing_dates(uk_projects_with_website_info)
uk_projects_with_website_info[['timeline', 'start_date', 'website_start_date', 'end_date', 'website_end_date']]

### Use Project Summaries from Website where Missing 

In [ ]:
[
    uk_projects_with_website_info.project_summary.isna().sum(),
    uk_projects_with_website_info.project_summary_website.isna().sum()
]

In [ ]:
def convert_project_summary_to_text(input_html):
    soup = BeautifulSoup(input_html, 'html5lib')
    output_text = soup.get_text().strip()
    output_text = re.sub(r'^Project Summary\s*', '', output_text)
    return output_text

def use_summary_from_website_where_missing(frame):
    is_missing = frame.project_summary.isna()
    frame.loc[is_missing, 'project_summary'] = \
        frame.project_summary_website[is_missing].apply(convert_project_summary_to_text)
use_summary_from_website_where_missing(uk_projects_with_website_info)
uk_projects_with_website_info.head()

## Import postcode data and check imported data

In [ ]:
ukpostcodes = pd.read_csv('../postcodes/input/ukpostcodes.csv.gz')

In [ ]:
ukpostcodes.shape

In [ ]:
uk_partnerships.raw_postcode.isin(ukpostcodes.postcode).sum()

In [ ]:
uk_partnerships['postcode'] = uk_partnerships.raw_postcode.\
    str.upper().\
    str.strip().\
    str.replace(r'^UK ', '').\
    str.replace(r'[^A-Z0-9]', '').\
    str.replace(r'^(\S+)([0-9][A-Z]{2})$', r'\1 \2')

In [ ]:
uk_partnerships.postcode.isin(ukpostcodes.postcode).sum()

In [ ]:
uk_partnerships.raw_postcode[~uk_partnerships.postcode.isin(ukpostcodes.postcode)].unique()

Typos, deprecated postcodes, one mistake, and one prefixed with UK

In [ ]:
clean_uk_partnerships = uk_partnerships[uk_partnerships.postcode.isin(ukpostcodes.postcode)].copy()
clean_uk_partnerships.shape

In [ ]:
clean_uk_projects = uk_projects_with_website_info[
    uk_projects_with_website_info.project_id.isin(clean_uk_partnerships.project_id)
].copy()
clean_uk_projects.shape

## Convert Currencies

In [ ]:
eur_gbp = pd.read_pickle('../exchange_rates/output/exchange_rates.pkl.gz')
eur_gbp.tail()

In [ ]:
def find_average_eur_gbp_rate(row):
    # create timeseries from start to end
    days = pd.date_range(row.start_date, row.end_date, closed='left')
    daily = pd.DataFrame({
        'month_start': days,
        'weight': 1.0 / days.shape[0]
    })
    monthly = daily.resample('MS', on='month_start').sum()
    monthly = pd.merge(monthly, eur_gbp, on='month_start', validate='1:1')
    return (monthly.weight * monthly.rate).sum()

clean_uk_projects['eur_gbp'] = clean_uk_projects.apply(
    find_average_eur_gbp_rate, axis=1, result_type='reduce')

In [ ]:
clean_uk_projects.head()

## Drop uninteresting columns

In [ ]:
clean_uk_partnerships_to_save = clean_uk_partnerships[[
    'project_id', 'partner_number', 'beneficiary', 'postcode',
    'contribution_eur', 'total_cost_eur'
]]
clean_uk_partnerships_to_save.head()

In [ ]:
clean_uk_projects_to_save = clean_uk_projects[[
    'project_id', 'project', 'project_summary',
    'start_date', 'end_date',
    'project_contribution_eur', 'project_total_cost_eur',
    'num_countries', 'num_partners',
    'image_path', 'page_path', 'eur_gbp'
]]

In [ ]:
clean_uk_projects_to_save.head()

In [ ]:
clean_uk_partnerships_to_save.to_pickle('output/partnerships.pkl.gz')

In [ ]:
clean_uk_projects_to_save.to_pickle('output/projects.pkl.gz')

In [ ]:
clean_uk_partnerships.postcode.sort_values().unique()